In [2]:
from AHI_AC import *
res = 0.01

row_AHI = round(120/res)
col_AHI = round(120/res)

AHI_lat = np.linspace(60 - res/2 ,-60 + res/2,row_AHI)
AHI_lon = np.linspace(85 + res/2 ,205 - res/2,col_AHI)

target ='/data01/GEO/SRDATA/'

AHI_VZA,AHI_VAA = read_AHI_VA(res)
Landmask = read_Landmask(res)
AHI_AL = read_AHI_AL(res)
Water_idx = get_water_idx(Landmask)
interpolation = LUT_interpolator(1)
fn1_1, fn2_1, fn3_1 = interpolation._interpolate()
interpolation = LUT_interpolator(2)
fn1_2, fn2_2, fn3_2 = interpolation._interpolate()
interpolation = LUT_interpolator(3)
fn1_3, fn2_3, fn3_3 = interpolation._interpolate()
interpolation = LUT_interpolator(4)
fn1_4, fn2_4, fn3_4 = interpolation._interpolate()

In [5]:
def SR_Process(band):
    start_time = T.time()

    # Download AHI
    print(f"Start download AHI for Band{band}")
    AHI_data = H8_Process('ACCOUNT', 'PW', band_type[band], band_num[band], date, folder_original)
    if AHI_data == 'No data':
        print(f"No AHI data for Band{band}, pass")
        return None
    E_Read_AHI_time = T.time()
    print(f"Finish download AHI for Band{band}")

    # SR calculate
    print(f'Start Calculate SR for Band{band}')
    S_SR_time = T.time()
    RESULT = Parallel(n_jobs=njob)(delayed(calculate_6s[band])(fn1[band], fn2[band], fn3[band], Landmask, Aerosol_type, *params[band], AHI_data, i) for i in range(12000))
    SR = np.full((12000, 12000), np.nan)
    for i in range(12000):
        SR[i][Water_idx[i]] = RESULT[i]
    E_SR_time = T.time()
    print(f'Calculate SR for Band{band} finished')

    # Save file and remove download input data
    print(f'Start Save SR file for Band{band}')
    S_SAVE_time = T.time()
    SR = np.array(SR).reshape(12000, 12000)
    SR = np.around(SR, 4) * 10000

    SR = np.where(SR >= 32767, 32767, SR)
    SR = np.where(SR <= -32767, -32767, SR)
    SR = np.where(np.isnan(SR), -32768, SR)

    SR_file = open(folder_AC + f'H08_{{YYYY}}{{MM}}{{DD}}_{{HH}}{{MIN}}_B{band}_SR.dat', 'wb')
    SR.astype('int16').tofile(SR_file)
    SR_file.close()

    print(f'Save Band{band} SR finished')
    # remove_original_file(folder_original)
    # print("delete file finish")
    E_SAVE_time = T.time()
    end_time = T.time()
    TIME = end_time - start_time
    print('Band{} cost time: {:.1f} secs, {:.1f} mins, {:.1f} hours\n\n'.format(band, TIME, TIME / 60, TIME / 3600))


# Set necessary parameters
band_type = {'1': 'vis', '2': 'vis', '3': 'ext', '4': 'vis'}
band_num = {'1': '01', '2': '02', '3': '01', '4': '03'}
calculate_6s = {'1': calculate_6s_band1, '2': calculate_6s_band2, '3': calculate_6s_band3, '4': calculate_6s_band4}
fn1 = {'1': fn1_1, '2': fn1_2, '3': fn1_3, '4': fn1_4}
fn2 = {'1': fn2_1, '2': fn2_2, '3': fn2_3, '4': fn2_4}
fn3 = {'1': fn3_1, '2': fn3_2, '3': fn3_3, '4': fn3_4}


In [12]:
njob = 15
date_start = '2018-12-23 00:00'
date_end = '2018-12-27 23:50'
date_t = dt.timedelta(minutes=10)
date_s = dt.datetime.strptime(date_start, "%Y-%m-%d %H:%M")
date_e = dt.datetime.strptime(date_end, "%Y-%m-%d %H:%M")
date_time_now = date_s
print("Start {} ".format(date_start[:-6]))
YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)
# CAMS_ds= CAMS_data(YYYY,MM,DD,HH,MIN,AHI_lat,AHI_lon,row_AHI).read_CAMS()
# CAMS_AERO_ds= CAMS_data(YYYY,MM,DD,HH,MIN,AHI_lat,AHI_lon,row_AHI).read_CAMS_AERO()
while date_time_now <= date_e:

    date_dl_str = date_time_now.strftime("%Y-%m-%d %H:%M" )
    YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)   
    hour = int(date[8:10])

    if hour >=2 and hour <= 10:
        print("start processing {}".format(date))
    # make dir
        S_time = T.time()
        folder_original = target + date +'_original/'
        folder_AC = target+'{}/{}/{}/'.format(YYYY,MM,DD)
        mkdir(folder_original)
        mkdir(folder_AC)

        # Solar angle
        print('Start read solar angle')
        S_ANGLE_time = T.time()
        AHI_SZA,AHI_SAA = AHI_angle(date,row_AHI).read_solar_angle()
        RAA = abs(AHI_SAA - AHI_VAA)
        RAA[RAA>180]=360-RAA[RAA>180]
        print('Finish read solar angle')
        E_ANGLE_time = T.time()
        # print('Read angle cost : {}'.format(E_ANGLE_time-S_ANGLE_time))
        
        # Atmosphereic data
        print('Start read CAMS')
        S_ATMOS_time = T.time()
        CAMS_data_obj = CAMS_data(YYYY, MM, DD, HH, MIN, AHI_lat, AHI_lon, row_AHI)
        Aerosol_type, OZ, WV, AOT550 = read_cams_data(CAMS_data_obj)
        print('Finish read CAMS')
        E_ATMOS_time = T.time()
        print('Read Atmospheric data cost : {}'.format(E_ATMOS_time-S_ATMOS_time))
        
        params = {
                    '1': [OZ, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                    '2': [OZ, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                    '3': [WV, OZ, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                    '4': [WV, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL]
                }
        
        for band in ['1', '2', '3', '4']:
            SR_Process(band)
        E_time = T.time()
        print('Total time cost : {}'.format(E_time-S_time))
    date_time_now = date_time_now + date_t

Start 2018-12-23 
start processing 201812230200
Start read solar angle
Finish read solar angle
Start read CAMS
Finish read CAMS
Read Atmospheric data cost : 9.002818584442139
Start download AHI for Band1
Finish download AHI for Band1
Start Calculate SR for Band1


<ipython-input-5-9e4eda4be726>:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if AHI_data == 'No data':


Calculate SR for Band1 finished
Start Save SR file for Band1
Save Band1 SR finished
Band1 cost time: 94.4 secs, 1.6 mins, 0.0 hours


Start download AHI for Band2
Finish download AHI for Band2
Start Calculate SR for Band2
Calculate SR for Band2 finished
Start Save SR file for Band2
Save Band2 SR finished
Band2 cost time: 66.4 secs, 1.1 mins, 0.0 hours


Start download AHI for Band3
Finish download AHI for Band3
Start Calculate SR for Band3
Calculate SR for Band3 finished
Start Save SR file for Band3
Save Band3 SR finished
Band3 cost time: 114.0 secs, 1.9 mins, 0.0 hours


Start download AHI for Band4
Finish download AHI for Band4
Start Calculate SR for Band4
Calculate SR for Band4 finished
Start Save SR file for Band4
Save Band4 SR finished
Band4 cost time: 56.1 secs, 0.9 mins, 0.0 hours


Total time cost : 342.0088403224945
start processing 201812230210
Start read solar angle
Finish read solar angle
Start read CAMS
Finish read CAMS
Read Atmospheric data cost : 29.523436307907104
Start

<ipython-input-5-9e4eda4be726>:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if AHI_data == 'No data':


Finish download AHI for Band1
Start Calculate SR for Band1
Calculate SR for Band1 finished
Start Save SR file for Band1
Save Band1 SR finished
Band1 cost time: 55.3 secs, 0.9 mins, 0.0 hours


Start download AHI for Band2
Finish download AHI for Band2
Start Calculate SR for Band2
Calculate SR for Band2 finished
Start Save SR file for Band2
Save Band2 SR finished
Band2 cost time: 56.8 secs, 0.9 mins, 0.0 hours


Start download AHI for Band3
Finish download AHI for Band3
Start Calculate SR for Band3



KeyboardInterrupt

